In [30]:
# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app
with open('yelp_api_auth', 'r') as authfile:
    API_KEY = authfile.readlines()[0]  # API Key is first line

In [44]:
# -*- coding: utf-8 -*-
"""
Yelp Fusion API code sample.
Currently only takes:
Location
Query Terms

and prints info for the top result business (no sorting order) and also reviews.

This program demonstrates the capability of the Yelp Fusion API
by using the Search API to query for businesses by a search term and location,
and the Business API to query additional information about the top result
from the search query.
Please refer to http://www.yelp.com/developers/v3/documentation for the API
documentation.
Sample usage of the program:
`python sample.py --term="bars" --location="San Francisco, CA"`
"""
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib


# This client code can run on Python 2.x or 3.x.  Your imports can be
# simpler if you only need one of those.
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode


# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
REVIEW_PATH = '/reviews'


def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))
    print(url_params)
    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

def make_term_location_params(term, location):
    """make JSON for a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON params.
    """
    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),  # can do latitude and longitude (2 params) instead
        # can also specify radius in meters, optionally (max 40,000, about 25 miles).
        'limit': SEARCH_LIMIT
    }
    
    return url_params


def search(api_key, url_params):
    """Query the Search API by given params.
    Args:
        url_params (dict): The search terms to pass to the API.
    Returns:
        dict: The JSON response from the request.
    """
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def get_business(api_key, business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)


def get_business_reviews(api_key, business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_rev_path = BUSINESS_PATH + business_id + REVIEW_PATH

    return request(API_HOST, business_rev_path, api_key)


def query_api(term, location):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, make_term_location_params(term, location))

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(API_KEY, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)

    print('Querying reviews from the top business')
    response = get_business_reviews(API_KEY, business_id)
    pprint.pprint(response, indent=2)
    
    print('Querying similar businesses to the top business')
    response = get_nearby_similar_businesses(API_KEY, "c0WpyZFR3EoEBKcoY2LZ3Q", 40000)
    pprint.pprint(response, indent=2)
    

In [26]:
# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 3  # if unspecified, will return 20. Max is 50

In [47]:
query_api(DEFAULT_TERM, DEFAULT_LOCATION)

Querying https://api.yelp.com/v3/businesses/search ...
{'term': 'dinner', 'location': 'San+Francisco,+CA', 'limit': 3}
3 businesses found, querying business info for the top result "c0WpyZFR3EoEBKcoY2LZ3Q" ...
Querying https://api.yelp.com/v3/businesses/c0WpyZFR3EoEBKcoY2LZ3Q ...
{}
Result for business "c0WpyZFR3EoEBKcoY2LZ3Q" found:
{ 'alias': 'marlowe-san-francisco-2',
  'categories': [ {'alias': 'newamerican', 'title': 'American (New)'},
                  {'alias': 'brasseries', 'title': 'Brasseries'}],
  'coordinates': {'latitude': 37.7783200510611, 'longitude': -122.396771288052},
  'display_phone': '(415) 777-1413',
  'hours': [ { 'hours_type': 'REGULAR',
               'is_open_now': True,
               'open': [ { 'day': 0,
                           'end': '2200',
                           'is_overnight': False,
                           'start': '1130'},
                         { 'day': 1,
                           'end': '2200',
                           'is_overnight'

In [46]:
def get_nearby_similar_businesses(api_key, business_id, radius):
    #In progress - for some reason currently not returning anything
    """
    Get businesses that are similar to a given business within a certain radius.
    
    Args:
        business_id (str): The ID of the business to seed the search
        radius (int): The radius in meters to restrict the search
    Returns:
        dict: The JSON response from the request.
    """
    seed_business = get_business(API_KEY, business_id)
    url_params = {
        'latitude': seed_business['coordinates']['latitude'],
        'longitude': seed_business['coordinates']['longitude'],
        'radius': radius,
        'categories': ','.join([cat['alias'] for cat in seed_business['categories']]),
        'limit': SEARCH_LIMIT
    }
    response = search(api_key, url_params)

In [43]:
print('Querying similar businesses to the top business')
response = get_nearby_similar_businesses(API_KEY, "c0WpyZFR3EoEBKcoY2LZ3Q", 60000)
pprint.pprint(response, indent=2)

Querying similar businesses to the top business
Querying https://api.yelp.com/v3/businesses/c0WpyZFR3EoEBKcoY2LZ3Q ...
{'latitude': 37.7783200510611, 'longitude': -122.396771288052, 'limit': 3}
Querying https://api.yelp.com/v3/businesses/search ...
None
